In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Предсказание target значения
## Импорт данных
### Первым делом в нашей работе будет импорт данных. Импортировать данные мы будем с помощью команды read_csv у библиотеки pandas. Для данных создадим две переменные: train_df, test_df. Также в нашем распоряжении будет пример того, как мы должны будем сохранить наш ответ в системе kaggle. 

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')                      # Переменная для обучения
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')                        # Переменная для тестирования
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv') # Пример того, как должен быть оформлен ответ

## Анализ данных
### После импорта данных нужно проверить наши датасеты на соответствие описанию. Сделаем это с помощью команды head() - первые значения (по умолчанию = 5)

In [ ]:
train_df.head()        # Первые пять значений тренировочного датасета

In [ ]:
test_df.head()         # Первые пять значений тестового значения

In [ ]:
sample_submission.head()  # Первые пять значений примерного ответа 

### Далее нужно проверить типы данных в наших датасетах. Это можно узнать с помощью команды info

In [ ]:
train_df.info()       # Неполная информация о тренировочном датасете 

In [ ]:
test_df.info()      # Неполная информация о тестовом датасете 

### Как видно, в наших датасетах полный набор данных, пропусков нигде не видно, типы данных приведены верно, а в тестовой таблице отсутствует target значения. Данные в идеальном состоянии, необходимо лишь преобразовать категорийные данные в числовые

## Преобразование данных
### У нас имеются данные категорийного типа, которые будут мешать нашему алгоритму работать, ведь работа с текстовой инфомрацией довольно муторная и долгая. Поэтому нам нужно преобразовать их в численные данные, а делать это мы будем с помощью one hot encoding

In [ ]:
all_df = pd.concat([train_df[train_df.columns[:-1]], test_df])
y = train_df['target']

In [ ]:
#categorical_df = all_df[all_df.columns[1:20]].copy()
#categorical_df.head()

categorical_columns = all_df.columns[1:20]
categorical_columns

In [ ]:
for column in categorical_columns:
    print(all_df[column].value_counts().count())

In [ ]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column, drop_first = True)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [ ]:
# all_dummies = []
# for column in categorical_columns:
#      all_dummies.append(onehot_encode(all_df,column))
        
for column in categorical_columns:
    all_df = onehot_encode(all_df, column)

In [ ]:
all_df.head()

In [ ]:
train_df = all_df[:len(train_df)]
test_df = all_df[len(train_df):]

train_df.shape, test_df.shape

## Обучение модели
### Перед тем как начать обучение моделей, мы должны разделить данные командой train_test_split, которую мы импортируем из skle

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df, y, test_size = 0.2, random_state = 451) # Делим данные на обучающие и предсказываемые

In [ ]:
X_train.head()                 # Первые пять значений тренировочных обучающих данных

In [ ]:
y_train.head()                 # Первые пять значений тренировочных предсказываемых данных

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score
from xgboost import XGBClassifier

### Сейчас мы будем обучать модель и смотреть качество предсказаний. Для более гибкой работы создадим цикл, который будет перебирать наши модели 
1. Логистическая регрессия
2. Дерево решений (классификация)
3. Случайный лес (классификация)
### В качестве метрики качества будем использовать accuracy score.
### Та модель, что даст нам лучший результат будет выписана в отедльную строку вместе со своим результатом

In [ ]:
models = [LogisticRegression(),
         DecisionTreeClassifier(),
         RandomForestClassifier(),
         GradientBoostingClassifier(),
         XGBClassifier()]

best_model = ''
best_score = -100

for model in models:
    
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    scores = accuracy_score(y_test, predictions)
    print(f"The model: {model}has following score: {scores}")
    
    if scores > best_score:
        best_score = scores
        best_model = model
        
print(f"\nThe best model is {best_model} it has the {best_score} scores")

### Как видно, лучший результат показал случайный лес - 84.7 процентов качества. Результат отличный, осталось попытаться его улучшить. Будем вводить random_state и n_estimators

In [ ]:
best_model = RandomForestClassifier(random_state = 451, n_estimators = 300)
best_model.fit(X_train, y_train)
predictions = best_model.predict(X_test)
scores = accuracy_score(y_test, predictions)
scores

### К сожалению наша модель показала незначительный прирост в 0.02 процента, что не очень практично, однако результат есть результат. Теперь нам нужно дать нашей модели тестовые данные и записать ответ

In [ ]:
test_predictions = best_model.predict(test_df)
test_predictions

### Ответ получился в виде списка, но нам нужно сохранить ответ в датафрейме, при этом с указанием id. Для этого воспользуемся командой DataFrame, которая превратит наши данные в датафрейм

In [ ]:
prediction_df = pd.DataFrame(data = test_predictions)
prediction_df.head()

### Отлично, ответ записан в виде DataFrame, но осталось добавить к нему id. Ну тут просто копируем 

In [ ]:
prediction_df['id'] = test_df['id'].copy()
prediction_df.head(15)

### Не порядок, наши id стоят в конце, хотя по примеру должны стоять впереди. Но это можно исправить с помощью команды tolist и дальнейших преобразований

In [ ]:
cols = prediction_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
prediction_df = prediction_df[cols]
prediction_df.head()

### И вот, наш ответ готов и его можно смело отправлять на проверку